In [ ]:
import pandas as pd
import pymc as pm
import arviz as az
import numpy as np

In [ ]:
data = pd.read_csv("real_estate_data.csv")

X1 = data['Surface_area'].values  
X2 = data['Rooms'].values         
X3 = data['Distance_to_center'].values  
y = data['Price'].values          

X1_norm = (X1 - np.mean(X1)) / np.std(X1)
X2_norm = (X2 - np.mean(X2)) / np.std(X2)
X3_norm = (X3 - np.mean(X3)) / np.std(X3)
y_norm = (y - np.mean(y)) / np.std(y)

In [ ]:
with pm.Model() as model:
    # Priori pentru coeficienti si intercept
    intercept = pm.Normal("intercept", mu=0, sigma=10) # Valoarea de baza a pretului 
    beta1 = pm.Normal("beta1", mu=0, sigma=10)  # Coeficient pentru X1 
    beta2 = pm.Normal("beta2", mu=0, sigma=10)  # Coeficient pentru X2 
    beta3 = pm.Normal("beta3", mu=0, sigma=10)  # Coeficient pentru X3 
    sigma = pm.HalfNormal("sigma", sigma=1)     # Deviatia standard pentru erorile de predictie
    
    # Modelul de regresie liniara
    mu = intercept + beta1 * X1_norm + beta2 * X2_norm + beta3 * X3_norm

    y_obs = pm.Normal("y_obs", mu=mu, sigma=sigma, observed=y_norm)
    
    trace = pm.sample(1000, tune=500, return_inferencedata=True)

In [ ]:
# Denormalizarea coeficientilor si interceptului
# Calcularea statisticilor variabilelor originale
mean_X1, std_X1 = np.mean(X1), np.std(X1)
mean_X2, std_X2 = np.mean(X2), np.std(X2)
mean_X3, std_X3 = np.mean(X3), np.std(X3)
mean_y, std_y = np.mean(y), np.std(y)

# Extragem coeficientii normalizati din model
beta1_norm = trace.posterior['beta1'].mean().item()
beta2_norm = trace.posterior['beta2'].mean().item()
beta3_norm = trace.posterior['beta3'].mean().item()
intercept_norm = trace.posterior['intercept'].mean().item()

# Calcularea coeficientilor denormalizati
beta1_original = beta1_norm * std_X1 + mean_X1
beta2_original = beta2_norm * std_X2 + mean_X2
beta3_original = beta3_norm * std_X3 + mean_X3

# Calcularea interceptului denormalizat
intercept_original = (
    intercept_norm * std_y
    + mean_y
    - (beta1_original * mean_X1 + beta2_original * mean_X2 + beta3_original * mean_X3)
)

In [ ]:
print("\nCoeficienti in unitati originale:")
print(f"Coeficientul pentru suprafata utila: {beta1_original:.2f} mii euro/m²")
print(f"Coeficientul pentru numarul de camere: {beta2_original:.2f} mii euro/camera")
print(f"Coeficientul pentru distanta fata de centru: {beta3_original:.2f} mii euro/km")
print(f"Intercept: {intercept_original:.2f} mii euro")

az.plot_posterior(trace, var_names=["beta1", "beta2", "beta3", "intercept"], hdi_prob=0.95)